### Download NWM Transport NetCDF files from S3
using fsspec and dask

In [ ]:
import pandas as pd
import fsspec
import dask
from dask.distributed import Client

In [ ]:
nwm_bucket = 's3://noaa-nwm-retro-v2.0-pds'
nwm_type = 'CHRTOUT'
ldir = '/home/jovyan/data'

In [ ]:
fs = fsspec.filesystem('s3', anon=True)

In [ ]:
dates = pd.date_range(start='2018-01-01 00:00',end='2018-01-03 23:00', freq='1h')

In [ ]:
@dask.delayed
def get_NWM(date):
    s3file = '{}/full_physics/{}/{}.{}_DOMAIN1.comp'.format(nwm_bucket,date.strftime('%Y'),
                date.strftime('%Y%m%d%H%M'),nwm_type)
    ofile = '{}/{}.{}_DOMAIN1.comp'.format(ldir,date.strftime('%Y%m%d%H%M'),nwm_type) 
    fs.download(s3file,ofile)

In [ ]:
tasks = [get_NWM(date) for date in dates]

In [ ]:
from dask_gateway import Gateway

from dask.distributed import Client

gateway = Gateway()
cluster = gateway.new_cluster(environment='pangeo', profile='Small Worker')

In [ ]:
client = Client(cluster)

In [ ]:
cluster.scale(2)

In [ ]:
%%time
with dask.config.set({"distributed.scheduler.allowed-failures": 10}):
    dask.compute(tasks);